In [34]:
from pandas import set_option as pd_set_option
pd_set_option('display.max_rows', 500)
pd_set_option('display.max_columns', 500)
from excel_app import getDataFromExcel
from pandas import concat as pd_concat
from pandas import merge as pd_merge
from pandas import MultiIndex as pd_MultiIndex
from excel_app import df_to_excel
from reports.contracts.contract import  get_xls_struct

def get_contract_report():
    xls_struct = get_xls_struct()
    df_period = getDataFromExcel("files//periods.xlsx", list(xls_struct[0]))
    df_period = df_period.rename(columns=xls_struct[0])
    df_rev = getDataFromExcel("files//revenew.xlsx", list(xls_struct[1]))
    df_rev = df_rev.rename(columns=xls_struct[1])
    df_exp = getDataFromExcel("files//expense.xlsx", list(xls_struct[2]))
    df_exp = df_exp.rename(columns=xls_struct[2])

    """union and merge table"""
    df_result = pd_concat([df_rev, df_exp], ignore_index=True, sort=False)
    # df_result = df_result.loc[df_result['rev_treat'] == '"Договор 00000000052 от 25.01.2012 13:03:37"']
    df_result = pd_merge(df_result, df_period, on=["treat", "branch"], how="left")
    df_result = df_result.fillna(
        {"date": "01.01.2019 0:00:00", "pay_plan": 0, "pay_fact": 0, "revenue_plan": 0, "revenue_fact": 0}).fillna(
        '')
    """building pivot table"""
    df_result = df_result.pivot_table(index=xls_struct[3], columns="date",
                                      values=["pay_plan", "pay_fact", "revenue_plan", "revenue_fact"], fill_value='',
                                      margins=True,
                                      aggfunc=sum).replace(0, '')
    """removing the last row with totals"""
    df_result = df_result[:-1]
    """to swipe columns"""
    # result = result.swaplevel(0, 1, axis=1).sort_index(axis=1)
    """reseting and building a new hierarchy indexes"""
    df_result = df_result.reset_index()
    df_result = df_result.set_index(['rev_treat', 'exp_treat']).sort_index()
    
    return df_result

get_contract_report()

branch  \
date                                                                                                               
rev_treat                                    exp_treat                                                             
"Договор 00000000052 от 25.01.2012 13:03:37"                                               Центральный офис (ЦО)   
                                             "Договор 00000000172 от 13.02.2012 10:48:20"  Центральный офис (ЦО)   
                                             "Договор 00000001258 от 27.03.2013 12:00:00"  Центральный офис (ЦО)   
                                             "Договор 00000003205 от 23.08.2015 12:00:01"  Центральный офис (ЦО)   
                                             "Договор 00000004572 от 27.09.2016 12:00:00"  Центральный офис (ЦО)   
                                             "Договор 00000004885 от 29.03.2017 12:55:32"  Центральный офис (ЦО)   
                                             "Договор 00000005015 от 25.05.2017 0:00:00"   Центральный офис (ЦО)   
                                             "Договор 00000005886 от 12.10.2018 0:00:00"   Центральный офис (ЦО)   
"Договор 00000000660 от 29.12.2012 9:57:11"                                                Центральный офис (ЦО)   
                                             "Договор 00000001385 от 26.06.2013 12:00:00"  Центральный офис (ЦО)   
                                             "Договор 00000005814 от 04.09.2018 0:00:00"   Центральный офис (ЦО)   
"Договор 00000000991 от 22.08.2012 13:59:39"                                               Центральный офис (ЦО)   
                                             "Договор 00000001571 от 12.09.2013 12:00:00"  Центральный офис (ЦО)   
                                             "Договор 00000004401 от 22.09.2016 12:00:00"  Центральный офис (ЦО)   
                                             "Договор 00000005239 от 20.07.2017 0:00:00"   Центральный офис (ЦО)   
"Договор 00000003129 от 29.05.2015 12:00:00"                                               Центральный офис (ЦО)   
                                             "Договор 00000003156 от 08.06.2015 12:00:01"  Центральный офис (ЦО)   
                                             "Договор 00000003761 от 25.03.2016 12:00:00"  Центральный офис (ЦО)   
                                             "Договор 00000003765 от 25.03.2016 12:00:01"  Центральный офис (ЦО)   
"Договор 00000003212 от 24.07.2015 12:00:00"                                               Центральный офис (ЦО)   
                                             "Договор 00000003228 от 24.07.2015 12:00:04"  Центральный офис (ЦО)   
"Договор 00000003466 от 23.06.2015 12:00:01"                                               Центральный офис (ЦО)   
                                             "Договор 00000003525 от 23.06.2015 12:00:05"  Центральный офис (ЦО)   
"Договор 00000003477 от 24.07.2015 12:00:13"                                               Центральный офис (ЦО)   
                                             "Договор 00000005459 от 20.01.2018 0:00:00"   Центральный офис (ЦО)   
"Договор 00000003515 от 07.08.2015 12:00:08"                                               Центральный офис (ЦО)   
                                             "Договор 00000004408 от 05.08.2016 0:00:00"   Центральный офис (ЦО)   
"Договор 00000003551 от 09.09.2015 12:00:00"                                               Центральный офис (ЦО)   
                                             "Договор 00000003553 от 09.09.2015 12:00:01"  Центральный офис (ЦО)   
"Договор 00000003587 от 23.06.2015 12:00:09"                                               Центральный офис (ЦО)   
                                             "Договор 00000003535 от 23.06.2015 12:00:08"  Центральный офис (ЦО)   
"Договор 00000003593 от 23.06.2015 12:00:12"                                               Центральный офис (ЦО)   
                                             "